In [1]:
import numpy as np
import gzip
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

C:\Users\sport\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def read_data(train_num_images,test_num_images,image_size,train_image_file,train_label_file,test_image_file,test_label_file):
    
    train_image_file = gzip.open(train_image_file,'r')
    train_label_file = gzip.open(train_label_file,'r')
    test_image_file = gzip.open(test_image_file,'r')
    test_label_file = gzip.open(test_label_file,'r')

    # Read Training Images
    train_image_file = gzip.open('train-images-idx3-ubyte.gz','r')
    train_image_file.read(16)
    buf = train_image_file.read(image_size * image_size * train_num_images)
    train_images = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    train_images = train_images.reshape(train_num_images, image_size, image_size, 1)

    # Read Training Labels
    train_label_file.read(8)
    buf = train_label_file.read(train_num_images)
    train_labels = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)

    # Read Testing Images
    test_image_file.read(16)
    buf = test_image_file.read(image_size * image_size * test_num_images)
    test_images = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    test_images = test_images.reshape(test_num_images, image_size, image_size, 1)

    # Read Testing Labels
    test_label_file.read(8)
    buf = test_label_file.read(test_num_images)
    test_labels = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    
    return train_images,train_labels,test_images,test_labels

In [3]:
def hyperparameter_tuning(x_train,y_train,x_test,y_test):
    iter = [10,20,30,40,50,75,100,150,200]
    opt_iter = 1
    max_score = 0
    for i in iter:
        logisticRegr = LogisticRegression(solver = 'lbfgs', max_iter = i, multi_class = 'multinomial', n_jobs=1, tol=0.0001, verbose=0)
        logisticRegr.fit(x_train, y_train)
        train_score = logisticRegr.score(x_train, y_train)
        val_score = logisticRegr.score(x_test, y_test)
        print("Maximum Iterations:{}, Train Accuracy:{:2.4}, Validation Accuracy:{:2.4}".format(i,100*train_score,100*val_score))
        if(val_score> max_score):
            max_score = val_score
            opt_iter = i
    print("Optimum:", opt_iter, 100*max_score)
    
    return opt_iter

In [4]:
def main():
    train_num_images = 60000
    test_num_images = 10000
    image_size = 28
    
    train_image_file = 'train-images-idx3-ubyte.gz'
    train_label_file = 'train-labels-idx1-ubyte.gz'
    test_image_file = 't10k-images-idx3-ubyte.gz'
    test_label_file = 't10k-labels-idx1-ubyte.gz'
    train_images,train_labels,test_images,test_labels = read_data(train_num_images,test_num_images,image_size,\
                                                                  train_image_file,train_label_file,test_image_file,\
                                                                  test_label_file)
    # Reshaping Images for training and testing
    train_images = train_images.reshape(train_num_images,784)
    test_images = test_images.reshape(test_num_images,784)
    
    # Splitting training data into trainig and validation data
    x_train, x_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.25, random_state=0)

    # Finding optimum number of iterations
    opt_iter = hyperparameter_tuning(x_train,y_train,x_test,y_test)
    
    # Training on the whole training dataset
    logisticRegr = LogisticRegression(solver = 'lbfgs', max_iter = opt_iter, multi_class = 'multinomial', n_jobs=1, tol=0.0001, verbose=0)
    logisticRegr.fit(train_images, train_labels)
    train_score = logisticRegr.score(train_images, train_labels)
    print("Train Accuracy:{:2.4}".format(100*train_score))

    predictions = logisticRegr.predict(test_images)
    test_score = logisticRegr.score(test_images, test_labels)
    print("Test Accuracy:{:2.4}".format(100*test_score))
    
    
    # Dumping testing results into csv file
    results = to_categorical(predictions,num_classes = 10)
    np.savetxt("lr.csv", results, fmt = '%4d', delimiter=",")

In [5]:
if __name__ == "__main__":
    main()

(60000, 784)
(60000,)
(45000, 784)
(15000, 784)
Maximum Iterations:10, Train Accuracy:88.69, Validation Accuracy:88.66
Maximum Iterations:20, Train Accuracy:91.15, Validation Accuracy:90.87
Maximum Iterations:30, Train Accuracy:92.13, Validation Accuracy:91.69
Maximum Iterations:40, Train Accuracy:92.52, Validation Accuracy:92.11
Maximum Iterations:50, Train Accuracy:92.74, Validation Accuracy:91.97
Maximum Iterations:75, Train Accuracy:93.23, Validation Accuracy:92.12
Maximum Iterations:100, Train Accuracy:93.52, Validation Accuracy:91.95
Maximum Iterations:150, Train Accuracy:93.79, Validation Accuracy:91.83
Maximum Iterations:200, Train Accuracy:94.02, Validation Accuracy:91.69
Optimum: 75 92.12
Train Accuracy:93.18
Test Accuracy:92.38
